
<p><img align="left" src="https://www.cqf.com/themes/custom/creode/logo.svg" style="vertical-align: top; padding-top: 23px;" width="10%"/>
<img align="right" src="https://upload.wikimedia.org/wikipedia/commons/c/c3/Python-logo-notext.svg" style="vertical-align: bottom;" width="12%"/>
<font color="#306998"><h1><center>Python Labs</center></h1></font></p>
<p></p><h1><center>Introduction to Financial Time Series</center></h1>
<center><b>Kannan Singaravelu</b></center>
<center>kannan.singaravelu@fitchlearning.com</center>



<h2 id="Financial-Data-Preprocessing">Financial Data Preprocessing<a name="top"></a><a class="anchor-link" href="#Financial-Data-Preprocessing">¶</a></h2><p>A time series is a series of data points indexed in time order. Financial data such as stock price series observed at equally spaced points in time are an example of such a series. It is a sequence of observations recorded at regular time intervals and depending on the frequency of observations, a time series may typically be in seconds, minute, hourly, daily, weekly, monthly, quarterly and annual.</p>
<p><img align="center" src="http://datascience.la/wp-content/uploads/2014/09/data-science-workflow-szilard.png" style="vertical-align: top; padding-top: 23px;" width="60%"/></p>
<p><b></b></p><center>Diagrammatic representation of a typical quant workflow.</center>
<p>The first step towards any data analysis would be to parse the raw data which involves extracting the data from the source. Data can be mined from Eikon API, Bloomberg, Google and Yahoo Finance. Data is then cleaned and missing values, if any are filled before performing feature scaling. The cleaned and transformed data is then fed into the model to validate the hypothesis.</p>
<p>While data comes in many forms, Python makes it easy to read time series data using useful packages. We will use <code>yfinance</code> and <code>alpha_vantage</code> libraries to retrieve historical end-of-day and intraday data for our purpose. These libraries aims to keep the API simple and makes it easier to access historical data. Further, we will see how to read data from CSV and Excel files stored locally.</p>
<p>Note: To run all of the code cells in this example, select <strong><em>Run All</em></strong> from the <strong><em>Cell</em></strong> menu.</p>



<h3 id="Installing-and-Importing-Required-Libraries">Installing and Importing Required Libraries<a class="anchor-link" href="#Installing-and-Importing-Required-Libraries">¶</a></h3><p>We'll install the required libraries that we'll use in this example.</p>


In [ ]:

# ! pip install yfinance
# ! pip install alpha-vantage




<p>We'll import the required libraries that we'll use in this example.</p>


In [ ]:

# Import required libraries
import pandas as pd
import numpy as np

# Import data libraries
import yfinance as yf

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')




<h3 id="Retrieving-End-of-Day-Data">Retrieving End-of-Day Data<a class="anchor-link" href="#Retrieving-End-of-Day-Data">¶</a></h3>



<blockquote><p><strong>A note on Pandas Datareader</strong><br/>
<em>For years, the go-to tool for downloading data from Yahoo Finance was the <code>pandas-datareader</code> library. However, this functionality was deprecated after some changes to the Yahoo Finance API. It still work in older version with limited capabilities. One should however get familiar with pandas-datareader as it facilitates downloading data from sources such as FRED (Federal Reserve Economic Data), the Fama/French Data Library or the World Bank.</em></p>
</blockquote>



<h4 id="Retrieving-Data-for-a-Single-Security">Retrieving Data for a Single Security<a class="anchor-link" href="#Retrieving-Data-for-a-Single-Security">¶</a></h4>



<p>We'll retrieve historical data from yahoo finance using <code>yfinance</code> library</p>



<h5 id="Example-1">Example 1<a class="anchor-link" href="#Example-1">¶</a></h5>


In [ ]:

yf.download?



In [ ]:

# Fetch data by specifying the number of the period
yf.download('SPY', period='5d')




<h5 id="Example-2">Example 2<a class="anchor-link" href="#Example-2">¶</a></h5>


In [ ]:

# Fetch the data from yahoo using start and end dates
yf.download('SPY', start='2020-06-20' , end='2020-06-30', progress=False)




<h5 id="Example-3">Example 3<a class="anchor-link" href="#Example-3">¶</a></h5>


In [ ]:

# Fetch data for year to date (YTD)
spy = yf.download('SPY', period='ytd', progress=False)



In [ ]:

# Display the first five rows of the dataframe to check the results. 
spy.head()




<h4 id="Retrieving-Data-for-Multiple-Securities">Retrieving Data for Multiple Securities<a class="anchor-link" href="#Retrieving-Data-for-Multiple-Securities">¶</a></h4><p>We'll retrieve daily historical price data of FAANG stocks from Yahoo Finance.</p>


In [ ]:

# Define the parameters
faang_stocks = ['AAPL', 'AMZN', 'FB', 'GOOG', 'NFLX']
start_date = '2019-06-01'
end_date = '2020-06-30'



In [ ]:

# Fetching data for multiple tickers
df = yf.download(faang_stocks, start=start_date, end=end_date, progress=False)['Adj Close']

# Display the first five rows of the dataframe to check the results. 
df.head()



In [ ]:

# Let's save the data for future use
df.to_csv('data/faang_stocks_1.csv')



In [ ]:

# # Display the first five rows of the dataframe to check the results. 
# df = df['2013':] #FB listed from mid 2012
# df.head()




<h4 id="Fetching-OHLCV-Data-for-Multiple-Securities">Fetching OHLCV Data for Multiple Securities<a class="anchor-link" href="#Fetching-OHLCV-Data-for-Multiple-Securities">¶</a></h4>


In [ ]:

{x**2 for x in range(6)}



In [ ]:

# Use comprehension to fetch data for multiple fields 
ohlc_data = {symbol: yf.download(symbol, start='2020-06-01', end='2020-06-30', progress=False) for symbol in faang_stocks}



In [ ]:

ohlc_data



In [ ]:

# Display AMZN stock data
ohlc_data['AMZN'].head()



In [ ]:

# Display AMZN Adjusted Close data
ohlc_data['AMZN']['Adj Close'].head()



In [ ]:

# df.dtypes
# df.index
# df.info() # dataframe metadata information




<h3 id="Retrieving-Intraday-Data">Retrieving Intraday Data<a class="anchor-link" href="#Retrieving-Intraday-Data">¶</a></h3>



<h4 id="Fetching-Intraday-Data-from-Yahoo-Finance">Fetching Intraday Data from Yahoo Finance<a class="anchor-link" href="#Fetching-Intraday-Data-from-Yahoo-Finance">¶</a></h4><p>We'll now retrieve intraday data from yahoo finance using <code>yfinance</code> library</p>


In [ ]:

# Retrive intraday data
dfi = yf.download(tickers="SPY", period="5d", interval="1m", progress=False)

# Display the first five rows of the dataframe to check the results.
dfi.head()



In [ ]:

dfi.tail()




<h4 id="Fetching-Intraday-Data-from-Alpha-Vantage">Fetching Intraday Data from Alpha Vantage<a class="anchor-link" href="#Fetching-Intraday-Data-from-Alpha-Vantage">¶</a></h4><p>We can also use <code>alpha_vantage</code> library to retrieve intraday timeseries data</p>


In [ ]:

from alpha_vantage.timeseries import TimeSeries



In [ ]:

key_path = "Key.txt"
ts = TimeSeries(key=open(key_path, 'r').read(),output_format='pandas')
data, metadata  = ts.get_intraday(symbol='AMZN',interval='1min', outputsize='full')



In [ ]:

metadata



In [ ]:

data




<h3 id="File-Input-&amp;-Output">File Input &amp; Output<a class="anchor-link" href="#File-Input-&amp;-Output">¶</a></h3>



<h4 id="Storing-OHLCV-data-in-Excel-File">Storing OHLCV data in Excel File<a class="anchor-link" href="#Storing-OHLCV-data-in-Excel-File">¶</a></h4>


In [ ]:

# Dataframe to Excel
from pandas import ExcelWriter



In [ ]:

# Storing the fetched data in a separate sheet for each security
writer = ExcelWriter('data/mystocks_1.xlsx')

[pd.DataFrame(ohlc_data[symbol]).to_excel(writer,symbol) for symbol in faang_stocks]

writer.save() # save file




<h4 id="Reading-Microsfot-Excel-File">Reading Microsfot Excel File<a class="anchor-link" href="#Reading-Microsfot-Excel-File">¶</a></h4><p>We'll now read the Excel file stored locally using Pandas</p>


In [ ]:

# Using get_history function to retriev India Volatility Index time series for a specific dates
faang = pd.read_excel('data/mystocks_1.xlsx', sheet_name='NFLX',index_col=0, parse_dates=True)

# Display the last five rows of the data frame to check the results
faang.tail()




<h4 id="Storing-OHLCV-data-in-a-CSV-File">Storing OHLCV data in a CSV File<a class="anchor-link" href="#Storing-OHLCV-data-in-a-CSV-File">¶</a></h4>


In [ ]:

# Save ohlcv data for each securities in stockname.csv format
[pd.DataFrame(ohlc_data[symbol]).to_csv('data/'+symbol+'.csv') for symbol in faang_stocks]
print('*** data saved ***')




<h4 id="Reading-CSV-File">Reading CSV File<a class="anchor-link" href="#Reading-CSV-File">¶</a></h4><p>We'll now read the csv file stored locally using Pandas</p>


In [ ]:

# Using get_history function to retriev Cboe Volatility Index time series for a specific dates
vix = pd.read_csv('data/vix.csv', index_col=0, parse_dates=True, dayfirst=True) 

# Display the last five rows of the data frame to check the results
vix.tail()




<h3 id="Resampling">Resampling<a class="anchor-link" href="#Resampling">¶</a></h3><p>Next, we'll manipulate the data retrieved by resampling the frequency of time series. This is very critical if you work on financial data or time series. For complete list of time period options when resampling time series, refer <a href="https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html">https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html</a></p>


In [ ]:

# Resampling to derive weekly values fro daily time series
spy_weekly = spy.resample('W').last()

# Resampling to derive monthly values from daily time series
spy_monthly = spy.resample('M').last()

# Display the first five rows of the data frame to check the output
spy_weekly.tail(5)




<h4 id="Weekly-Resample-on-Thursday">Weekly Resample on Thursday<a class="anchor-link" href="#Weekly-Resample-on-Thursday">¶</a></h4>


In [ ]:

spy.index



In [ ]:

spy_weekly_thu = spy.resample('W-THU').ffill()
spy_weekly_thu.tail()




<h2 id="Interactive-Visualization">Interactive Visualization<a class="anchor-link" href="#Interactive-Visualization">¶</a></h2><p>We use <code>cufflinks</code> for interactive visualization. It is one of the most feature rich third-party wrapper around Plotly by Santos Jorge. It binds the power of <code>plotly</code> with the flexibility of <code>pandas</code> for easy plotting.</p>
<p>When you import cufflinks library, all pandas data frames and series objects have a new method <code>.iplot()</code> attached to them which is similar to pandas' built-in <code>.plot()</code> method.</p>


In [ ]:

# Import cufflinks
import cufflinks as cf
cf.set_config_file(offline=True)




<h3 id="Visualising-a-Price-Series">Visualising a Price Series<a class="anchor-link" href="#Visualising-a-Price-Series">¶</a></h3><p>We'll now plot historical price series of CBOE Volatility Index using just one line of code.</p>


In [ ]:

df['AAPL'].iplot(kind='line', title='CBOE Volatily Index')




<h3 id="Plotting-OHLC-Data">Plotting OHLC Data<a class="anchor-link" href="#Plotting-OHLC-Data">¶</a></h3><p>Next, we'll plot the time series data in ohlc format.</p>


In [ ]:

spy[-30:].iplot(kind='ohlc',title='SPY Price')




<h3 id="Plotting-Candlestick">Plotting Candlestick<a class="anchor-link" href="#Plotting-Candlestick">¶</a></h3><p>Next, we'll plot an interactive candlestick chart.</p>


In [ ]:

spy[-30:].iplot(kind='candle', title='SPY Price')




<h3 id="Plotting-Selected-Stocks">Plotting Selected Stocks<a class="anchor-link" href="#Plotting-Selected-Stocks">¶</a></h3><p>Next, we'll compare the FAANG stocks data that we fetched from Yahoo Finance.</p>


In [ ]:

# Use secondary axis
df[['AMZN', 'AAPL']].iplot(title='Amazon Vs Apple',secondary_y='AAPL')




<h3 id="Plotting-using-Subplots">Plotting using Subplots<a class="anchor-link" href="#Plotting-using-Subplots">¶</a></h3>


In [ ]:

# Use subplots
df[['AMZN', 'AAPL']].iplot(title='Amazon Vs Apple Price Movement',subplots=True)




<h3 id="Normalized-Plot">Normalized Plot<a class="anchor-link" href="#Normalized-Plot">¶</a></h3>


In [ ]:

df.normalize().iplot(title='The FAANG Stocks')



In [ ]:

# Calculating Log Normal Returns
# Use numpy log function to derive log normal returns
daily_returns = np.log(df).diff().fillna(0)

# Display the last five rows of the data frame to check the output
daily_returns.head(5)




<h3 id="Visualising-Return-Series">Visualising Return Series<a class="anchor-link" href="#Visualising-Return-Series">¶</a></h3><p>We'll now plot historical daily log normal return series of NIFTY Index using just one line of code.</p>


In [ ]:

daily_returns[['AAPL','GOOG']].iplot(title='Daily Log Returns')




<h3 id="Plotting-Annual-Returns">Plotting Annual Returns<a class="anchor-link" href="#Plotting-Annual-Returns">¶</a></h3>


In [ ]:

# Mean Annual Returns
(daily_returns.mean()*252).iplot(kind='bar')




<h3 id="Rolling-Returns">Rolling Returns<a class="anchor-link" href="#Rolling-Returns">¶</a></h3>


In [ ]:

# To calculate 21 days rolling returns, simply sum daily returns for 22 days as log returns are additive
rolling_return = daily_returns.rolling(21).sum().dropna()

# Display the last five rows of the data frame to check the output
rolling_return.head(5)




<h3 id="Visualising-a-Rolling-Return-Series">Visualising a Rolling Return Series<a class="anchor-link" href="#Visualising-a-Rolling-Return-Series">¶</a></h3><p>We'll now plot 21-day rolling returns of AAPL using just one line of code.</p>


In [ ]:

rolling_return['AAPL'].iplot(title='21-Days Rolling Returns of Apple')




<h3 id="Normal-Distribution">Normal Distribution<a class="anchor-link" href="#Normal-Distribution">¶</a></h3><p>A normal distribution is the most common and widely used distribution in statistics. It is popularly referred as a “bell curve” or “Gaussian curve”. Financial time series though random in short term, follows a normal distribution on a longer time frame.</p>
<p>Now that we have derived the daily log returns, we will plot this return distribution.</p>


In [ ]:

# Plot log normal distribution of returns
daily_returns.iplot(kind='histogram', title = 'Histogram of Daily Returns', subplots=True)




<h3 id="Correlation">Correlation<a class="anchor-link" href="#Correlation">¶</a></h3><p>Correlation defines the similarity between two random variables. As an example we will check correlation between the FAANG stocks.</p>


In [ ]:

# Calculate correlation of returns
daily_returns.corr().iplot(kind='heatmap', title="Correlation Matrix", colorscale="Blues")




<h2 id="References">References<a class="anchor-link" href="#References">¶</a></h2><ul>
<li><p>YFinance documentation <a href="https://github.com/ranaroussi/yfinance">https://github.com/ranaroussi/yfinance</a></p>
</li>
<li><p>Alphavantage documentation <a href="https://github.com/RomelTorres/alpha_vantage">https://github.com/RomelTorres/alpha_vantage</a></p>
</li>
<li><p>Cufflinks documentation <a href="https://github.com/santosjorge/cufflinks">https://github.com/santosjorge/cufflinks</a> and <a href="https://plotly.com/python/cufflinks/">https://plotly.com/python/cufflinks/</a></p>
</li>
<li><p>Python resources <a href="https://github.com/kannansingaravelu/PythonResources">https://github.com/kannansingaravelu/PythonResources</a></p>
</li>
</ul>
